In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pima-indians-diabetes-database/diabetes.csv


In [2]:
demo = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")
demo.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
#Analyse Skews
#Analyse Pairplot (Infer whether its a easy or hard classification problem)

In [5]:
demo.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
sns.pairplot(demo,hue = 'Outcome',diag_kind='kde')

In [ ]:
demo.skew()

In [ ]:
#Fit StandardScaler here
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
sc=StandardScaler()
X = demo.drop('Outcome',axis=1)
y = demo[['Outcome']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=20)
scaledxtrain = sc.fit_transform(X_train)
scaledxtest = sc.transform(X_test)
lr = LogisticRegression()
lr.fit(scaledxtrain,y_train)
print("What is the Testing Accuracy")
print(lr.score(scaledxtest,y_test))
print("What is the Training Accuracy")
print(lr.score(scaledxtrain,y_train))

In [ ]:
sns.pairplot(X_train,diag_kind='kde')

In [ ]:
demo.info()

In [ ]:
import numpy as np
demo['Glucose'].replace(0, np.nan,inplace=True)
demo['BloodPressure'].replace(0, np.nan,inplace=True)
demo.info()

In [ ]:
#Missing Values with Iterative Imputer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PowerTransformer
X=demo.drop('Outcome',axis=1)
y=demo['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=20)
it=IterativeImputer()
itXtrain = pd.DataFrame(it.fit_transform(X_train))
itXtest = pd.DataFrame(it.transform(X_test))
sc=StandardScaler()
scaledxtrain = sc.fit_transform(itXtrain)
scaledxtest = sc.transform(itXtest)
lr = LogisticRegression()
lr.fit(scaledxtrain,y_train)
print("What is the Testing Accuracy")
print(lr.score(scaledxtest,y_test))
print("What is the Training Accuracy")
print(lr.score(scaledxtrain,y_train))

In [ ]:
#With Pipeline
from sklearn.pipeline import Pipeline
pipe = Pipeline((
("it", IterativeImputer()),
("sc", StandardScaler()),
("lr", LogisticRegression()),
))
pipe.fit(X_train,y_train)
print("Testing Accuracy")
print(pipe.score(X_test,y_test))
print("Training Accuracy")
print(pipe.score(X_train,y_train))

In [ ]:
predicted = pipe.predict(X_test)
from sklearn.metrics import confusion_matrix,classification_report,recall_score,precision_score,f1_score
print(confusion_matrix(y_test,predicted))
print(classification_report(y_test,predicted))